<a href="https://www.kaggle.com/code/siddp6/simple-skip-connection?scriptVersionId=138504859" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports

In [15]:
import utility

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Initalisation

In [6]:
input_size, output_size = 1024, 8
num_samples = 1000
epochs = 10
input_data = torch.rand(num_samples, input_size)
target_data = torch.rand(num_samples, output_size)
loss_normal = list()
loss_skip = list()

In [7]:
def plot_loss(losses_normal, losses_skip):
    plt.figure(figsize=(8, 5))
    
    plt.plot(range(1, len(losses_normal) + 1), losses_normal, label="Noraml")
    plt.plot(range(1, len(losses_skip) + 1), losses_skip, label="Skip")
    
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.legend()
    plt.grid(True)
    plt.show()

# Normal Model(without skip-connection)

In [8]:
class NormalNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NormalNet, self).__init__()
        
        self.layer_1 = nn.Linear(input_size, 512)
        self.layer_2 = nn.Linear(512, 256)
        self.layer_3 = nn.Linear(256, 128)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_6 = nn.Linear(32, 16)
        self.layer_7 = nn.Linear(16, output_size)
        
    def forward(self, X):
        X = F.relu(self.layer_1(X))
        X = F.relu(self.layer_2(X))
        X = F.relu(self.layer_3(X))
        X = F.relu(self.layer_4(X))
        X = F.relu(self.layer_5(X))
        X = F.relu(self.layer_6(X))
        X = self.layer_7(X)
        
        return X

In [9]:
model = NormalNet(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=.01)

In [10]:
loss_normal = utility.train_model(model, epochs, input_data, target_data, criterion, optimizer, 100)

# Skip Model(with skip-connection)

In [11]:
class SkipNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SkipNN, self).__init__()
        
        self.layer_1 = nn.Linear(input_size, 512)        
        self.layer_2 = nn.Linear(512, 256)
        self.layer_3 = nn.Linear(256, 128)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_6 = nn.Linear(32, 16)
        self.layer_7 = nn.Linear(16, output_size)
        
    def forward(self, X):
        X1 = F.relu(self.layer_1(X))
        X2 = F.relu(self.layer_2(X1)) + X1
        X3 = F.relu(self.layer_3(X2)) + X1
        X4 = F.relu(self.layer_4(X3)) + X1
        X5 = F.relu(self.layer_5(X4)) + X1
        X6 = F.relu(self.layer_6(X5)) + X1
        X7 = self.layer_7(X6)
        
        return X7

In [12]:
skip_model = NormalNet(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(skip_model.parameters(), lr=.01)

In [13]:
loss_skip = utility.train_model(skip_model, epochs, input_data, target_data, criterion, optimizer, 100)

# Analysis

In [17]:
utility.plot_loss({"Normal": loss_normal, "Skip": loss_skip}, epochs)

NameError: name 'plt' is not defined